In [236]:
inp_short = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k"""

In [237]:
def nested_set(dic, keys, value):
    for key in keys[:-1]:
        dic = dic.setdefault(key, {})
    if keys[-1] not in dic:
        dic[keys[-1]] = 0
    dic[keys[-1]] += value

In [242]:
def parse_cmds(inp: str) -> dict:
    curr_path = ""
    result = {}
    for line in inp.split("\n"):
        # Figure out current path
        line = line.strip()
        path_parts = [part for part in curr_path.split("/")]
        if line.startswith("$ cd .."):
            curr_path = "/".join(path_parts[:-1])
            if not curr_path:
                curr_path = "/"
        elif line.startswith("$ cd /"):
            curr_path = "/"
        elif line.startswith("$ cd "):
            curr_path = ("/".join(path_parts) + "/" + line[5:]).replace("//","/")
        # Record file-sizes in folders
        if line[0].isdigit():
            size = int(line.split(" ")[0])
            filename = line.split(" ")[-1]
            nested_set(result, 
                       ["/"] + [x for x in curr_path.split("/") if x] + ['files'], 
                       size)
    return result

In [243]:
structure = parse_cmds(inp_short)
structure

{'/': {'files': 23352670,
  'a': {'files': 94269, 'e': {'files': 584}},
  'd': {'files': 24933642}}}

In [352]:
def folder_sizes(struct: dict, folder_size = 0):
    for path, content in struct.items():
        if path != "files":
            if isinstance(content, dict):
                print(path)
                folder_size = folder_sizes(content, folder_size)
    print(struct)
    if "files" in struct.keys():
        return struct["files"] + folder_size
    else:
        return folder_size

In [353]:
folder_sizes(structure)

/
a
e
{'files': 584}
{'files': 94269, 'e': {'files': 584}}
d
{'files': 24933642}
{'files': 23352670, 'a': {'files': 94269, 'e': {'files': 584}}, 'd': {'files': 24933642}}
{'/': {'files': 23352670, 'a': {'files': 94269, 'e': {'files': 584}}, 'd': {'files': 24933642}}}


48381165